In [11]:
from dataclasses import dataclass
import numpy as np
import math
import statistics
import matplotlib.pyplot as plt
import import_ipynb

from statistics import mean
from ellipse import LsqEllipse
from matplotlib.patches import Ellipse

from loc3d import Loc3D

@dataclass
class Direction3D():
    dataname: str
    start: Loc3D
    stop: Loc3D

    def __repr__(self) -> str:
        """This function returns data type name"""
        return "Shift3D"

    def __str__(self) -> str:
        """This function returns a human-readable explpanation of what this class is"""
        
        return """This data represents the shift from the following set of points
                : {0} \n\nto\n\n {1}""".format(self.start.__repr__, self.stop.__repr__)

    def find_dir(self) -> list:
        
        """This function simply finds the distance between a list of points representing x & y & returns them in an array"""
        
        self.x_dir = np.array(self.stop.x) - np.array(self.start.x)
        self.y_dir = np.array(self.stop.y) - np.array(self.start.y)
        self.z_dir = np.array(self.stop.z) - np.array(self.start.z)
        
        self.distances = np.round(np.sqrt((np.array(self.stop.x) - np.array(self.start.x))**2 + (np.array(self.stop.y) - np.array(self.start.y))**2 + (np.array(self.stop.z) - np.array(self.start.z))**2),3)
    
        return self.x_dir, self.y_dir, self.z_dir, self.distances
    
    def find_dir_ls(self, start, stop) -> list:
        
        """This function simply finds the distance between a list of points representing x & y & returns them in an array"""
        
        x_dir = np.array(stop.x) - np.array(start.x)
        y_dir = np.array(stop.y) - np.array(start.y)
        z_dir = np.array(stop.z) - np.array(start.z)
    
        return x_dir, y_dir, z_dir
    
    def _merge_list(self,a, b):
        return list(map(lambda x, y:np.array([x,y]), a, b))
    
    def circle_data(self)-> np.array:
        self.R_data = np.array(self._merge_list(self.stop.x[0:5], self.stop.y[0:5]))
        self.S_data = np.array(self._merge_list(self.stop.x[5:10], self.stop.y[5:10]))
        return np.array(self.R_data), np.array(self.S_data)

    def quiver_plot (self, title="Quiver Plot", scale=0.5):
    
        """This function takes x & y positions & directions at arugments and generates a quiver plot overlayed 
            on top of an image"""
        
        self.x_dir, self.y_dir, self.z_dir, self.distances = self.find_dir()
        
        self.x_dir_mean = np.round(np.mean(self.x_dir),3)
        self.y_dir_mean = np.round(np.mean(self.y_dir),3)
        self.z_dir_mean = np.round(np.mean(self.z_dir),3)
        
        self.distance_mean = np.round(np.mean(self.distances),3)

        print(f'X-directions:\n{self.x_dir}\nX-direction Avg:\n{self.x_dir_mean}\n\nY-directions:\n{self.y_dir}\nY-direction Avg:\n{self.y_dir_mean}\n\nZ-directions:\n{self.z_dir}\nZ-direction Avg:\n{self.z_dir_mean}\n\nTotal Distances:\n{self.distances}\nAverage Total Distance:\n{self.distance_mean}')
        
        plt.close("all")
        
        fig, ax = plt.subplots(figsize = (20, 11))
        ax.quiver(self.start.x, self.start.y, self.x_dir, self.y_dir,
                scale = scale, color="C0")

        ax.axis([-75, 75, -75, 75])
        plt.title(title)
        ax.axhline(y=0, color='k')
        ax.axvline(x=0, color='k')

        # zip joins x and y coordinates in pairs
        for label,x,y in zip(self.start.labels, self.start.x, self.start.y):

            if label[-1] == "4" or label[-1] == "5":

                plt.annotate(label, # this is the text
                            (x,y), # these are the coordinates to position the label
                            textcoords="offset points", # how to position the text
                            xytext=(0,12), # distance from text to points (x,y)
                            ha='center') # horizontal alignment can be left, right or center
            elif label[-1] == "1":
                plt.annotate(label, # this is the text
                            (x,y), # these are the coordinates to position the label
                            textcoords="offset points", # how to position the text
                            xytext=(20, 0), # distance from text to points (x,y)
                            ha='center') # horizontal alignment can be left, right or center
            elif label[-1] == "2":
                plt.annotate(label, # this is the text
                            (x,y), # these are the coordinates to position the label
                            textcoords="offset points", # how to position the text
                            xytext=(0,-20), # distance from text to points (x,y)
                            ha='center') # horizontal alignment can be left, right or center
            elif label[-1] == "3":
                plt.annotate(label, # this is the text
                            (x,y), # these are the coordinates to position the label
                            textcoords="offset points", # how to position the text
                            xytext=(-15,-15), # distance from text to points (x,y)
                            ha='center') # horizontal alignment can be left, right or center
            elif label == "M":
                plt.annotate(label, # this is the text
                            (x,y), # these are the coordinates to position the label
                            textcoords="offset points", # how to position the text
                            xytext=(-15,-15), # distance from text to points (x,y)
                            ha='center') # horizontal alignment can be left, right or center
            elif label == "N":
                plt.annotate(label, # this is the text
                            (x,y), # these are the coordinates to position the label
                            textcoords="offset points", # how to position the text
                            xytext=(-5,15), # distance from text to points (x,y)
                            ha='center') # horizontal alignment can be left, right or center
            elif label == "Upper":
                plt.annotate(label, # this is the text
                            (x,y), # these are the coordinates to position the label
                            textcoords="offset points", # how to position the text
                            xytext=(20,-15), # distance from text to points (x,y)
                            ha='center') # horizontal alignment can be left, right or center
            elif label == "Lower":
                plt.annotate(label, # this is the text
                            (x,y), # these are the coordinates to position the label
                            textcoords="offset points", # how to position the text
                            xytext=(15,15), # distance from text to points (x,y)
                            ha='center') # horizontal alignment can be left, right or center

        """This section adds the engineering drawing to the background of the plot"""

        #This image was captured using a screenshot application and saved in the local 
        imagefile = "1000689_clean.jpg"

        img = plt.imread(imagefile)
        plt.imshow(img, zorder=0, extent=[-75, 75, -75, 75]) # we match the image extent to the limits of the axes
        
        plt.savefig(title + ".jpg")

    def ls_circle(self):
        lsqe = LsqEllipse()
        lsqe.fit(self.plot_data)
        center, width, height, phi = lsqe.as_parameters()
        
        return center, mean([width,height]), statistics.stdev([width,height])

    def ls_circle_plot(self, title="Least-Squares Circle Plot", scale=0.5):
    
        imagefile = "1000689_clean.jpg"
        img = plt.imread(imagefile)
        
        X_START = [0]
        Y_START = [0]
        Z_START = [0]
        
        ORIGIN = Loc3D(X_START, Y_START, Z_START)
        
        self.circle_data()
        
        # ---- R Data ---- #
        lsqe_R = LsqEllipse()
        lsqe_R.fit(np.array(self.R_data))
        center_R, width_R, height_R, phi_R = lsqe_R.as_parameters()        
        
        x_end_R, y_end_R, z_end_R = list(center_R)[0], list(center_R)[1], [0]
        
        CENTER_R = Loc3D(x_end_R, y_end_R, z_end_R)
        
        self.x_dir_center_R, self.y_dir_center_R, self.z_dir_center_R = self.find_dir_ls(ORIGIN, CENTER_R)
        
        # ---- S Data ---- #
        lsqe_S = LsqEllipse()
        lsqe_S.fit(np.array(self.S_data))
        center_S, width_S, height_S, phi_S = lsqe_S.as_parameters()
        
        x_end_S, y_end_S, z_end_S = list(center_S)[0], list(center_S)[1], [0]
        
        CENTER_S = Loc3D(x_end_S, y_end_S, z_end_S)
        
        self.x_dir_center_S, self.y_dir_center_S, self.z_dir_center_S = self.find_dir_ls(ORIGIN, CENTER_S)

        # -------- Plotting --------#
        
        # Close any hanging displays
        plt.close('all')
        
        # Create the figure with subplots and axes that represent the two subplots
        fig, ax = plt.subplots(2,1, figsize=(20,20))
        
        # Setting the limits of first subplot
        ax[0].axis([-75, 75, -75, 75])
        
        # Setting the title of the first subplot
        ax[0].title.set_text("R-Cones Least-Squares Circle")
        
        # Add the background image to the plot covering the full limits
        ax[0].imshow(img, zorder=0, extent=[-75, 75, -75, 75])
        
        # Setting the limits of second subplot
        ax[1].axis([-75, 75, -75, 75])
        
        # Setting the title of the second subplot
        ax[1].title.set_text("S-Cones Least-Squares Circle")
        
        # Add the background image to the plot covering the full limits
        ax[1].imshow(img, zorder=0, extent=[-75, 75, -75, 75]) # we match the image extent to the limits of the axes
        
        # Quiver Plot | R Cones
        ax[0].quiver(X_START, Y_START, self.x_dir_center_R, self.y_dir_center_R,
                scale = scale, color="C0")
        
        # Quiver Plot | R Cones
        ax[1].quiver(X_START, Y_START, self.x_dir_center_S, self.y_dir_center_S,
                scale = scale, color="C0")
        
        # Data Plot | R Cones
        ax[0].plot(self.R_data[:,0], self.R_data[:,1], 'ro', label='test data', zorder=1)
        
        # Data Plot | S Cones
        ax[1].plot(self.S_data[:,0], self.S_data[:,1], 'ro', label='test data', zorder=1)

        # Circle Plot | R
        ellipse_R = Ellipse(xy=center_R, width=2*width_R, height=2*height_R, angle=np.rad2deg(phi_R),
                    edgecolor='b', fc='None', lw=2, label='Fit', zorder = 2)
        ax[0].add_patch(ellipse_R)
        
        # Circle Plot | R
        ellipse_S = Ellipse(xy=center_S, width=2*width_S, height=2*height_S, angle=np.rad2deg(phi_S),
                    edgecolor='b', fc='None', lw=2, label='Fit', zorder = 2)
        ax[1].add_patch(ellipse_S)
        
        # Adding text to first plot
        ax[0].text(-68, 63, f'center of fitted circle =')
        ax[0].text(-68, 58, f'{np.round(center_R, 3)}')
        ax[0].text(-68, 53, f'radius =')
        ax[0].text(-68, 48, f'{np.round(mean([width_R,height_R]), 3)} +/- stddev = {np.round(statistics.stdev([width_R,height_R]),4)}')
        
        # Adding text to second plot
        ax[1].text(-68, 63, f'center of fitted circle =')
        ax[1].text(-68, 58, f'{np.round(center_S, 3)}')
        ax[1].text(-68, 53, f'radius =')
        ax[1].text(-68, 48, f'{np.round(mean([width_S,height_S]), 3)} +/- stddev = {np.round(statistics.stdev([width_S,height_S]),4)}')
        
        
        # Save the figure
        plt.savefig(self.dataname + " Least-Squares Circle.jpg")
        
        print('center of fitted circle =',np.round(center_R, 3), '\n','radius =', np.round(mean([width_R,height_R]), 3),
            '+/- stddev=',
            np.round(statistics.stdev([width_R,height_R]), 4))
        
        print('center of fitted circle =',np.round(center_S, 3), '\n','radius =', np.round(mean([width_S,height_S]), 3),
            '+/- stddev=',
            np.round(statistics.stdev([width_S,height_S]), 4))
        
        plt.legend()
        plt.show()